In [ ]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [ ]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [ ]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [ ]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

In [ ]:
df.columns

# Select your features (columns)

In [ ]:
# Set features. This will also be used as your x values. feature importance was used to select the more effective ones.
selected_features = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 
        'koi_impact',
        'koi_duration',
        'koi_depth', 
        'koi_prad', 
       'koi_teq', 'koi_insol', 
       'koi_model_snr'
         
         
       ]]

# Create a Train Test Split

Use `koi_disposition` for the y values

In [ ]:
# define the output as target
target = df["koi_disposition" ]


In [ ]:
selected_features_dummies = pd.get_dummies(selected_features)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(selected_features_dummies, target, random_state=42)


In [ ]:
 y_train.head()

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [ ]:
# Scale your data

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
X_scaler = MinMaxScaler().fit(X_train)


X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)



In [ ]:
# Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [ ]:
# Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train_scaled, y_train_categorical)
rf.score(X_test_scaled, y_test_categorical)

In [ ]:
predicted = rf.predict(X_test_scaled)



In [ ]:
# check the importance of the features and remove the least important ones
sorted(zip(rf.feature_importances_, selected_features_dummies), reverse=True)

# Train the Model



In [ ]:
print(f"Training Data Score: {rf.score(X_train_scaled, y_train_categorical)}")
print(f"Testing Data Score: {rf.score(X_test_scaled, y_test_categorical)}")

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [ ]:
# Create the GridSearchCV model

from sklearn.model_selection import GridSearchCV
param_grid = {'min_samples_split': [2, 5, 10],
              'bootstrap': [ True],
             'n_estimators': [100]}
grid = GridSearchCV(rf, param_grid, verbose=3)

In [ ]:
# Train the model with GridSearch.Several different parameters were tested
grid.fit(X_train_scaled, y_train_categorical)

In [ ]:
# rf.get_params().keys()

In [ ]:
print(grid.best_params_)
print(grid.best_score_)

In [ ]:
rf = grid

In [ ]:
rf.predict (X_test_scaled)

# Save the Model

In [ ]:
# save your model 
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'fatmabutun_rf.sav'
joblib.dump(rf, filename)